# Cowork-Style Tools — Interactive Test Notebook

# Setup

In [1]:
import sys
sys.path.append("..")

In [2]:
import base64
import os
import shutil
import tempfile
import time
import urllib.request
from pathlib import Path

# Create isolated temp directory for all tests
TEMP_DIR = Path(tempfile.mkdtemp(prefix="cowork_tool_tests_"))
print("TEMP_DIR:", TEMP_DIR)

# --- Seed text files ---
(TEMP_DIR / "README.md").write_text(
    "# Test Project\n\nThis is a test README.\n\n"
    "## Section A\nContent A here.\n\n## Section B\nContent B here.\n"
)

(TEMP_DIR / "test.txt").write_text("line one\nline two\nline three\n")

sub_dir = TEMP_DIR / "subdir"
sub_dir.mkdir()
(sub_dir / "nested.md").write_text("# Nested\nNested content here.\n")
(sub_dir / "data.txt").write_text("alpha\nbeta\ngamma\n")

(TEMP_DIR / "empty.txt").touch()

(TEMP_DIR / "long_lines.txt").write_text(
    f"short line\n{'X' * 3000}\nanother short line\n"
)

(TEMP_DIR / "no_trailing_newline.txt").write_text("first\nsecond\nthird")

print("Text files seeded.")

TEMP_DIR: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874
Text files seeded.


In [3]:
# --- Generate / download multimodal test assets ---

# PNG: 1x1 red pixel (base64-decoded, no network needed)
(TEMP_DIR / "test.png").write_bytes(base64.b64decode(
    "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlE"
    "QVR42mP8z8BQDwADhQGAWjR9awAAAABJRU5ErkJggg=="
))
print(f"Generated test.png ({(TEMP_DIR / 'test.png').stat().st_size} bytes)")

# GIF: 1x1 red pixel (base64-decoded, well-known minimal GIF89a)
(TEMP_DIR / "test.gif").write_bytes(base64.b64decode(
    "R0lGODlhAQABAIAAAP8AAAAAACH5BAAAAAAALAAAAAABAAEAAAICRAEAOw=="
))
print(f"Generated test.gif ({(TEMP_DIR / 'test.gif').stat().st_size} bytes)")

# PDF: minimal valid PDF (text-based, generated locally)
(TEMP_DIR / "test.pdf").write_bytes(
    b"%PDF-1.0\n"
    b"1 0 obj<</Type/Catalog/Pages 2 0 R>>endobj\n"
    b"2 0 obj<</Type/Pages/Kids[3 0 R]/Count 1>>endobj\n"
    b"3 0 obj<</Type/Page/MediaBox[0 0 3 3]>>endobj\n"
    b"xref\n0 4\n"
    b"0000000000 65535 f \n"
    b"0000000009 00000 n \n"
    b"0000000058 00000 n \n"
    b"0000000115 00000 n \n"
    b"trailer<</Size 4/Root 1 0 R>>\n"
    b"startxref\n190\n%%EOF"
)
print(f"Generated test.pdf ({(TEMP_DIR / 'test.pdf').stat().st_size} bytes)")

# JPEG + WebP: download from httpbin.org (complex to generate without PIL)
DOWNLOAD_ASSETS = {
    "test.jpg": ("https://httpbin.org/image/jpeg", "image/jpeg"),
    "test.webp": ("https://httpbin.org/image/webp", "image/webp"),
}
for fname, (url, accept) in DOWNLOAD_ASSETS.items():
    dest = TEMP_DIR / fname
    try:
        req = urllib.request.Request(url, headers={"Accept": accept})
        with urllib.request.urlopen(req, timeout=15) as resp:
            dest.write_bytes(resp.read())
        print(f"Downloaded {fname} ({dest.stat().st_size} bytes)")
    except Exception as e:
        print(f"Warning: Could not download {fname}: {e}")
        # Fallback: synthetic file with correct magic bytes
        if fname.endswith(".jpg"):
            dest.write_bytes(
                b"\xff\xd8\xff\xe0\x00\x10JFIF\x00"
                b"\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xd9"
            )
        elif fname.endswith(".webp"):
            dest.write_bytes(
                b"RIFF\x24\x00\x00\x00WEBPVP8 "
                b"\x18\x00\x00\x000\x01\x00\x9d\x01\x2a"
                b"\x01\x00\x01\x00\x01\x40\x25\xa4\x00"
                b"\x03p\x00\xfe\xfb\x94\x00\x00"
            )
        print(f"  -> Created synthetic {fname} ({dest.stat().st_size} bytes)")

print("\nAll test assets ready.")

Generated test.png (70 bytes)
Generated test.gif (43 bytes)
Generated test.pdf (285 bytes)
Downloaded test.jpg (35588 bytes)
Downloaded test.webp (10568 bytes)

All test assets ready.


# Tool Tests

## Tool: Read File

In [4]:
import json
from anthropic_agent.cowork_style_tools import create_read_tool

read_file = create_read_tool()
print("Tool name:", read_file.__tool_schema__["name"])
print("Schema:", json.dumps(read_file.__tool_schema__, indent=2))

Tool name: read_file
Schema: {
  "name": "read_file",
  "description": "Read a file from the local filesystem.\n\nReads text files and returns content with line numbers (cat -n format).\nFor images and PDFs, the file content is returned directly for the\nmodel to process visually.",
  "input_schema": {
    "type": "object",
    "properties": {
      "file_path": {
        "type": "string",
        "description": "Absolute path to the file to read."
      },
      "offset": {
        "type": [
          "integer",
          "null"
        ],
        "description": "Line number to start reading from (1-based). Defaults to 1."
      },
      "limit": {
        "type": [
          "integer",
          "null"
        ],
        "description": "Number of lines to read. Defaults to 2000."
      }
    },
    "required": [
      "file_path"
    ]
  }
}


### Text Files

In [5]:
# 1. Read full text file — verify cat-n line numbering
result = read_file(file_path=str(TEMP_DIR / "README.md"))
print("=== Read full text file ===")
print(result)
print()
assert "# Test Project" in result
# First line should be numbered "1\t..."
first_line = result.splitlines()[0]
assert first_line.strip().startswith("1")
print("PASS: Full text read with line numbering")

=== Read full text file ===
1	# Test Project
2	
3	This is a test README.
4	
5	## Section A
6	Content A here.
7	
8	## Section B
9	Content B here.

PASS: Full text read with line numbering


In [6]:
# 2. Read with offset and limit (windowed read)
result = read_file(file_path=str(TEMP_DIR / "README.md"), offset=3, limit=2)
print("=== Read with offset=3, limit=2 ===")
print(result)
lines = result.strip().splitlines()
assert len(lines) == 2, f"Expected 2 lines, got {len(lines)}"
assert "3" in lines[0].split("\t")[0]
assert "4" in lines[1].split("\t")[0]
print("\nPASS: Windowed read OK")

=== Read with offset=3, limit=2 ===
3	This is a test README.
4	

PASS: Windowed read OK


In [7]:
# 3-7. Error cases
print("=== Empty file ===")
result = read_file(file_path=str(TEMP_DIR / "empty.txt"))
print(result)
assert "Warning" in result and "empty" in result
print("PASS\n")

print("=== Non-existent file ===")
result = read_file(file_path=str(TEMP_DIR / "nonexistent.txt"))
print(result)
assert "Error" in result and "does not exist" in result
print("PASS\n")

print("=== Directory path ===")
result = read_file(file_path=str(TEMP_DIR / "subdir"))
print(result)
assert "Error" in result and "directory" in result.lower()
print("PASS\n")

print("=== Relative path ===")
result = read_file(file_path="relative/path.txt")
print(result)
assert "Error" in result and "absolute" in result.lower()
print("PASS\n")

print("=== Offset exceeding total lines ===")
result = read_file(file_path=str(TEMP_DIR / "test.txt"), offset=9999)
print(result)
assert "Error" in result and "offset" in result.lower()
print("PASS")

=== Empty file ===
PASS

=== Non-existent file ===
Error: File does not exist: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/nonexistent.txt
PASS

=== Directory path ===
Error: Path is a directory, not a file: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/subdir. Use ls via the Bash tool to list directory contents.
PASS

=== Relative path ===
Error: Path must be absolute, got relative path: relative/path.txt
PASS

=== Offset exceeding total lines ===
Error: offset (9999) exceeds total lines (3) in /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/test.txt
PASS


In [8]:
# 8. Long lines — truncation at 2000 chars
result = read_file(file_path=str(TEMP_DIR / "long_lines.txt"))
print("=== Long lines ===")
for i, line in enumerate(result.splitlines(), 1):
    print(f"Line {i} length: {len(line)}")
# The second line (3000 X's) should be truncated
line2 = result.splitlines()[1]
assert "[truncated]" in line2, "Expected truncation marker"
print("PASS: Long line truncation\n")

# 9. File without trailing newline
result = read_file(file_path=str(TEMP_DIR / "no_trailing_newline.txt"))
print("=== No trailing newline ===")
print(result)
lines = result.strip().splitlines()
assert len(lines) == 3, f"Expected 3 lines, got {len(lines)}"
assert "third" in lines[-1]
print("PASS: No trailing newline handled correctly")

=== Long lines ===
Line 1 length: 12
Line 2 length: 2014
Line 3 length: 20
PASS: Long line truncation

=== No trailing newline ===
1	first
2	second
3	third
PASS: No trailing newline handled correctly


### Image Files

In [9]:
from anthropic_agent.tools.base import ToolResult, ImageBlock, DocumentBlock

# 10. Read PNG image
result = read_file(file_path=str(TEMP_DIR / "test.png"))
print("=== Read PNG ===")
print(f"Return type: {type(result).__name__}")
assert isinstance(result, ToolResult), f"Expected ToolResult, got {type(result)}"
assert isinstance(result.content, list)
assert isinstance(result.content[0], str)  # text description
assert isinstance(result.content[1], ImageBlock)
assert result.content[1].media_type == "image/png"
print(f"Text: {result.content[0]}")
print(f"Media type: {result.content[1].media_type}")
print(f"Data size: {len(result.content[1].data)} bytes")
print("PASS: PNG read returns ToolResult with ImageBlock")

=== Read PNG ===
Return type: ToolResult
Text: Image file: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/test.png
Media type: image/png
Data size: 70 bytes
PASS: PNG read returns ToolResult with ImageBlock


In [10]:
# 11. Read JPEG image
result = read_file(file_path=str(TEMP_DIR / "test.jpg"))
print("=== Read JPEG ===")
print(f"Return type: {type(result).__name__}")
assert isinstance(result, ToolResult)
img_block = result.content[1]
assert isinstance(img_block, ImageBlock)
assert img_block.media_type == "image/jpeg"
print(f"Media type: {img_block.media_type}")
print(f"Data size: {len(img_block.data)} bytes")
print("PASS: JPEG read OK")

=== Read JPEG ===
Return type: ToolResult
Media type: image/jpeg
Data size: 35588 bytes
PASS: JPEG read OK


In [11]:
# 12. Read GIF image
result = read_file(file_path=str(TEMP_DIR / "test.gif"))
print("=== Read GIF ===")
print(f"Return type: {type(result).__name__}")
assert isinstance(result, ToolResult)
img_block = result.content[1]
assert isinstance(img_block, ImageBlock)
assert img_block.media_type == "image/gif"
print(f"Media type: {img_block.media_type}")
print(f"Data size: {len(img_block.data)} bytes")
print("PASS: GIF read OK")

=== Read GIF ===
Return type: ToolResult
Media type: image/gif
Data size: 43 bytes
PASS: GIF read OK


In [12]:
# 13. Read WebP image
result = read_file(file_path=str(TEMP_DIR / "test.webp"))
print("=== Read WebP ===")
print(f"Return type: {type(result).__name__}")
assert isinstance(result, ToolResult)
img_block = result.content[1]
assert isinstance(img_block, ImageBlock)
assert img_block.media_type == "image/webp"
print(f"Media type: {img_block.media_type}")
print(f"Data size: {len(img_block.data)} bytes")
print("PASS: WebP read OK")

=== Read WebP ===
Return type: ToolResult
Media type: image/webp
Data size: 10568 bytes
PASS: WebP read OK


### PDF Files

In [13]:
# 14. Read PDF file
result = read_file(file_path=str(TEMP_DIR / "test.pdf"))
print("=== Read PDF ===")
print(f"Return type: {type(result).__name__}")
assert isinstance(result, ToolResult)
assert isinstance(result.content, list)
assert isinstance(result.content[0], str)  # text description
assert isinstance(result.content[1], DocumentBlock)
assert result.content[1].media_type == "application/pdf"
print(f"Text: {result.content[0]}")
print(f"Media type: {result.content[1].media_type}")
print(f"Data size: {len(result.content[1].data)} bytes")
print("PASS: PDF read returns ToolResult with DocumentBlock")

=== Read PDF ===
Return type: ToolResult
Text: PDF document: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/test.pdf
Media type: application/pdf
Data size: 285 bytes
PASS: PDF read returns ToolResult with DocumentBlock


## Tool: Write File

In [14]:
from anthropic_agent.cowork_style_tools import create_write_tool

write_file = create_write_tool()
print("Tool name:", write_file.__tool_schema__["name"])

Tool name: write_file


In [15]:
# 1. Write a new file
new_file = str(TEMP_DIR / "written.txt")
result = write_file(file_path=new_file, content="Hello World\nLine 2\nLine 3\n")
print("=== Write new file ===")
print(result)
assert "Successfully wrote" in result
assert "3 lines" in result
# Verify content
actual = Path(new_file).read_text()
assert actual == "Hello World\nLine 2\nLine 3\n"
print("PASS: New file written correctly")

=== Write new file ===
Successfully wrote 3 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/written.txt
PASS: New file written correctly


In [16]:
# 2. Overwrite existing file
result = write_file(file_path=new_file, content="Replaced content\n")
print("=== Overwrite existing file ===")
print(result)
assert "Successfully wrote" in result
assert "1 lines" in result
actual = Path(new_file).read_text()
assert actual == "Replaced content\n", f"Got: {actual!r}"
print("PASS: File overwritten correctly")

=== Overwrite existing file ===
Successfully wrote 1 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/written.txt
PASS: File overwritten correctly


In [17]:
# 3. Write with auto-created nested parent directories
deep_file = str(TEMP_DIR / "a" / "b" / "c" / "deep.txt")
result = write_file(file_path=deep_file, content="Deep file\n")
print("=== Auto-create parent dirs ===")
print(result)
assert "Successfully wrote" in result
assert Path(deep_file).exists()
assert Path(deep_file).read_text() == "Deep file\n"
print("PASS: Parent directories auto-created")

=== Auto-create parent dirs ===
Successfully wrote 1 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/a/b/c/deep.txt
PASS: Parent directories auto-created


In [18]:
# 4. Write with relative path — expect error
result = write_file(file_path="relative.txt", content="nope")
print("=== Relative path error ===")
print(result)
assert "Error" in result and "absolute" in result.lower()
print("PASS: Relative path rejected")

=== Relative path error ===
Error: Path must be absolute, got relative path: relative.txt
PASS: Relative path rejected


In [19]:
# 5. Write empty content
empty_file = str(TEMP_DIR / "empty_written.txt")
result = write_file(file_path=empty_file, content="")
print("=== Write empty content ===")
print(result)
assert "0 lines" in result
assert Path(empty_file).read_text() == ""
print("PASS: Empty content, 0 lines\n")

# 6. Line count: content ending with newline vs without
f1 = str(TEMP_DIR / "with_newline.txt")
f2 = str(TEMP_DIR / "without_newline.txt")

r1 = write_file(file_path=f1, content="a\nb\nc\n")
r2 = write_file(file_path=f2, content="a\nb\nc")

print("=== Line count accuracy ===")
print(f"With trailing newline:    {r1}")
print(f"Without trailing newline: {r2}")
# "a\nb\nc\n" = 3 lines, "a\nb\nc" = 3 lines
assert "3 lines" in r1
assert "3 lines" in r2
print("PASS: Line count accurate for both cases")

=== Write empty content ===
Successfully wrote 0 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/empty_written.txt
PASS: Empty content, 0 lines

=== Line count accuracy ===
With trailing newline:    Successfully wrote 3 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/with_newline.txt
Without trailing newline: Successfully wrote 3 lines to /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/without_newline.txt
PASS: Line count accurate for both cases


## Tool: Edit File

In [20]:
from anthropic_agent.cowork_style_tools import create_edit_tool

edit_file = create_edit_tool()
print("Tool name:", edit_file.__tool_schema__["name"])

# Create a test file for edit operations
EDIT_FILE = str(TEMP_DIR / "editable.txt")
Path(EDIT_FILE).write_text("alpha bravo charlie\nalpha delta echo\nfoxtrot golf hotel\n")
print("Edit test file created:", EDIT_FILE)
print(Path(EDIT_FILE).read_text())

Tool name: edit_file
Edit test file created: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt
alpha bravo charlie
alpha delta echo
foxtrot golf hotel



In [21]:
# 1. Single unique replacement
result = edit_file(file_path=EDIT_FILE, old_string="foxtrot golf hotel", new_string="foxtrot GOLF hotel")
print("=== Single replacement ===")
print(result)
assert "Successfully edited" in result
assert "1 replacement" in result
assert "foxtrot GOLF hotel" in Path(EDIT_FILE).read_text()
print("PASS: Single replacement OK")

=== Single replacement ===
Successfully edited /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt. 1 replacement(s) made.
PASS: Single replacement OK


In [22]:
# 2. replace_all with multiple occurrences
# "alpha" appears on lines 1 and 2
result = edit_file(file_path=EDIT_FILE, old_string="alpha", new_string="ALPHA", replace_all=True)
print("=== replace_all ===")
print(result)
assert "Successfully edited" in result
assert "2 replacement" in result
content = Path(EDIT_FILE).read_text()
assert content.count("ALPHA") == 2
assert content.count("alpha") == 0
print("PASS: replace_all OK\n")
print("Current content:")
print(content)

=== replace_all ===
Successfully edited /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt. 2 replacement(s) made.
PASS: replace_all OK

Current content:
ALPHA bravo charlie
ALPHA delta echo
foxtrot GOLF hotel



In [23]:
# 3-9. Error cases
print("=== old_string not found ===")
result = edit_file(file_path=EDIT_FILE, old_string="NONEXISTENT", new_string="x")
print(result)
assert "Error" in result and "not found" in result
print("PASS\n")

print("=== Multiple occurrences without replace_all ===")
result = edit_file(file_path=EDIT_FILE, old_string="ALPHA", new_string="X")
print(result)
assert "Error" in result and "2 times" in result
print("PASS\n")

print("=== Empty old_string ===")
result = edit_file(file_path=EDIT_FILE, old_string="", new_string="x")
print(result)
assert "Error" in result and "empty" in result.lower()
print("PASS\n")

print("=== Identical old_string and new_string ===")
result = edit_file(file_path=EDIT_FILE, old_string="ALPHA", new_string="ALPHA")
print(result)
assert "Error" in result and "identical" in result.lower()
print("PASS\n")

print("=== Relative path ===")
result = edit_file(file_path="relative.txt", old_string="a", new_string="b")
print(result)
assert "Error" in result and "absolute" in result.lower()
print("PASS\n")

print("=== Non-existent file ===")
result = edit_file(file_path=str(TEMP_DIR / "no_such_file.txt"), old_string="a", new_string="b")
print(result)
assert "Error" in result and "does not exist" in result
print("PASS\n")

print("=== Path is a directory ===")
result = edit_file(file_path=str(TEMP_DIR / "subdir"), old_string="a", new_string="b")
print(result)
assert "Error" in result and "not a file" in result.lower()
print("PASS")

=== old_string not found ===
Error: old_string not found in /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt.
PASS

=== Multiple occurrences without replace_all ===
Error: old_string appears 2 times in /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt. Provide more context to make it unique, or set replace_all=true.
PASS

=== Empty old_string ===
Error: old_string cannot be empty.
PASS

=== Identical old_string and new_string ===
Error: old_string and new_string are identical. No changes needed.
PASS

=== Relative path ===
Error: Path must be absolute, got relative path: relative.txt
PASS

=== Non-existent file ===
Error: File does not exist: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/no_such_file.txt
PASS

=== Path is a directory ===
Error: Path is not a file: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/subdir
PASS


In [24]:
# 10. Multiline replacement
multiline_file = str(TEMP_DIR / "multiline_edit.txt")
Path(multiline_file).write_text("line 1\nold block start\nold block middle\nold block end\nline 5\n")

result = edit_file(
    file_path=multiline_file,
    old_string="old block start\nold block middle\nold block end",
    new_string="new block A\nnew block B",
)
print("=== Multiline replacement ===")
print(result)
assert "Successfully edited" in result
content = Path(multiline_file).read_text()
print(content)
assert "new block A\nnew block B" in content
assert "old block" not in content
print("PASS: Multiline replacement OK")

=== Multiline replacement ===
Successfully edited /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/multiline_edit.txt. 1 replacement(s) made.
line 1
new block A
new block B
line 5

PASS: Multiline replacement OK


## Tool: Glob Search

In [25]:
from anthropic_agent.cowork_style_tools import create_glob_tool

glob_search = create_glob_tool()
print("Tool name:", glob_search.__tool_schema__["name"])

Tool name: glob_search


In [26]:
# 1. Match by extension
result = glob_search(pattern="*.txt", path=str(TEMP_DIR))
print("=== Match *.txt ===")
print(result)
assert "test.txt" in result
print("PASS\n")

# 2. Recursive glob
result = glob_search(pattern="**/*.md", path=str(TEMP_DIR))
print("=== Recursive **/*.md ===")
print(result)
assert "README.md" in result
assert "nested.md" in result
print("PASS")

=== Match *.txt ===
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/multiline_edit.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/editable.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/without_newline.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/with_newline.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/empty_written.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/written.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/no_trailing_newline.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/long_lines.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/empty.txt
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y

In [27]:
# 3. No matches
result = glob_search(pattern="*.xyz", path=str(TEMP_DIR))
print("=== No matches ===")
print(result)
assert result == "No matches found."
print("PASS\n")

# 4. Non-existent path
result = glob_search(pattern="*.txt", path=str(TEMP_DIR / "nonexistent_dir"))
print("=== Non-existent path ===")
print(result)
assert "Error" in result
print("PASS\n")

# 5. Path is a file, not directory
result = glob_search(pattern="*.txt", path=str(TEMP_DIR / "test.txt"))
print("=== Path is a file ===")
print(result)
assert "Error" in result and "not a directory" in result.lower()
print("PASS")

=== No matches ===
No matches found.
PASS

=== Non-existent path ===
Error: Path does not exist: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/nonexistent_dir
PASS

=== Path is a file ===
Error: Path is not a directory: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/test.txt
PASS


In [28]:
# 6. Verify results are absolute paths
result = glob_search(pattern="*.md", path=str(TEMP_DIR))
print("=== Absolute paths ===")
for line in result.strip().splitlines():
    assert line.startswith("/"), f"Not absolute: {line}"
    print(f"  {line}")
print("PASS: All paths are absolute\n")

# 7. Verify sorted by mtime (newest first)
# Create files with staggered mtimes
import time as _time
for i, name in enumerate(["oldest.tmp", "middle.tmp", "newest.tmp"]):
    p = TEMP_DIR / name
    p.write_text(f"file {i}")
    _time.sleep(0.1)  # ensure different mtimes

result = glob_search(pattern="*.tmp", path=str(TEMP_DIR))
print("=== Sorted by mtime (newest first) ===")
print(result)
lines = result.strip().splitlines()
assert "newest.tmp" in lines[0], f"Expected newest first, got: {lines[0]}"
assert "oldest.tmp" in lines[-1], f"Expected oldest last, got: {lines[-1]}"
print("PASS: Sorted by mtime, newest first")

=== Absolute paths ===
  /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/README.md
PASS: All paths are absolute

=== Sorted by mtime (newest first) ===
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/newest.tmp
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/middle.tmp
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/oldest.tmp
PASS: Sorted by mtime, newest first


## Tool: Grep Search

In [29]:
from anthropic_agent.cowork_style_tools import create_grep_tool

grep_search = create_grep_tool()
print("Tool name:", grep_search.__tool_schema__["name"])

Tool name: grep_search


In [30]:
# 1. files_with_matches mode (default)
result = grep_search(pattern="alpha", path=str(TEMP_DIR))
print("=== files_with_matches mode ===")
print(result)
assert "data.txt" in result
print("PASS")

=== files_with_matches mode ===
/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/subdir/data.txt
PASS


In [31]:
# 2. content mode with line numbers
result = grep_search(pattern="line", path=str(TEMP_DIR / "test.txt"), output_mode="content")
print("=== content mode with line numbers ===")
print(result)
# Should show line numbers by default
assert "1:" in result or "1-" in result  # line number prefix
print("PASS\n")

# 3. content mode with context lines
result = grep_search(
    pattern="delta",
    path=str(TEMP_DIR / "editable.txt"),
    output_mode="content",
    context=1,
)
print("=== content mode with context=1 ===")
print(result)
print("PASS\n")

# 3b. after_context and before_context
result = grep_search(
    pattern="line two",
    path=str(TEMP_DIR / "test.txt"),
    output_mode="content",
    before_context=1,
    after_context=1,
)
print("=== content mode with before=1, after=1 ===")
print(result)
assert "line one" in result  # before context
assert "line three" in result  # after context
print("PASS")

=== content mode with line numbers ===
1:line one
2:line two
3:line three
PASS

=== content mode with context=1 ===
1-ALPHA bravo charlie
2:ALPHA delta echo
3-foxtrot GOLF hotel
PASS

=== content mode with before=1, after=1 ===
1-line one
2:line two
3-line three
PASS


In [32]:
# 4. count mode
result = grep_search(pattern="ALPHA", path=str(TEMP_DIR / "editable.txt"), output_mode="count")
print("=== count mode ===")
print(result)
assert "2" in result  # ALPHA appears 2 times (single file: no filename prefix)
print("PASS")

=== count mode ===
2
PASS


In [33]:
# 5. Case-insensitive search
result = grep_search(
    pattern="alpha",
    path=str(TEMP_DIR / "editable.txt"),
    output_mode="count",
    case_insensitive=True,
)
print("=== case_insensitive ===")
print(result)
# "ALPHA" appears 2 times; case-insensitive "alpha" should also match those
assert "2" in result  # single file: no filename prefix
print("PASS\n")

# 6. Regex pattern (not just literal)
result = grep_search(pattern="line (one|three)", path=str(TEMP_DIR / "test.txt"), output_mode="content")
print("=== Regex pattern ===")
print(result)
assert "line one" in result
assert "line three" in result
assert "line two" not in result
print("PASS\n")

# 7. include_glob filter
result = grep_search(pattern="content", path=str(TEMP_DIR), include_glob="*.md")
print("=== include_glob=*.md ===")
print(result)
# Should only match .md files, not .txt files
for line in result.strip().splitlines():
    assert line.endswith(".md"), f"Expected .md file, got: {line}"
print("PASS")

=== case_insensitive ===
2
PASS

=== Regex pattern ===
1:line one
3:line three
PASS

=== include_glob=*.md ===
/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/subdir/nested.md
PASS


In [34]:
# 8. Error cases
print("=== Empty pattern ===")
result = grep_search(pattern="")
print(result)
assert "Error" in result and "empty" in result.lower()
print("PASS\n")

print("=== Invalid output_mode ===")
result = grep_search(pattern="test", output_mode="invalid_mode")
print(result)
assert "Error" in result and "invalid" in result.lower()
print("PASS\n")

print("=== No matches ===")
result = grep_search(pattern="ZZZZUNMATCHABLE", path=str(TEMP_DIR))
print(result)
assert "No matches found" in result
print("PASS")

=== Empty pattern ===
Error: Search pattern cannot be empty.
PASS

=== Invalid output_mode ===
Error: Invalid output_mode 'invalid_mode'. Expected 'files_with_matches', 'content', or 'count'.
PASS

=== No matches ===
No matches found.
PASS


In [35]:
# 9. head_limit and offset
# First, get the full content result
full_result = grep_search(
    pattern=".",
    path=str(TEMP_DIR / "test.txt"),
    output_mode="content",
)
full_lines = full_result.strip().splitlines()
print(f"Full result: {len(full_lines)} lines")
print(full_result)

# With head_limit=1, should get only first line
limited = grep_search(
    pattern=".",
    path=str(TEMP_DIR / "test.txt"),
    output_mode="content",
    head_limit=1,
)
print(f"\n=== head_limit=1 ===")
print(limited)
limited_lines = limited.strip().splitlines()
assert len(limited_lines) == 1, f"Expected 1 line, got {len(limited_lines)}"
print("PASS\n")

# With offset=1, head_limit=1, should skip first and get second
offset_result = grep_search(
    pattern=".",
    path=str(TEMP_DIR / "test.txt"),
    output_mode="content",
    offset=1,
    head_limit=1,
)
print("=== offset=1, head_limit=1 ===")
print(offset_result)
assert offset_result.strip() != limited.strip(), "Offset should give different result"
print("PASS")

Full result: 3 lines
1:line one
2:line two
3:line three

=== head_limit=1 ===
1:line one
PASS

=== offset=1, head_limit=1 ===
2:line two
PASS


## Tool: Bash

In [36]:
from anthropic_agent.cowork_style_tools import BashTool

# Create a dedicated bash sandbox directory
BASH_DIR = TEMP_DIR / "bash_sandbox"
BASH_DIR.mkdir()

bash_tool = BashTool(base_path=str(BASH_DIR))
bash = bash_tool.get_tool()
print("Tool name:", bash.__tool_schema__["name"])
print("Base path:", bash_tool.base_path)
print("Sandbox enabled:", bash_tool.sandbox_enabled)

Tool name: bash
Base path: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox
Sandbox enabled: True


In [37]:
# 1. Basic command execution
result = bash(command="echo hello world")
print("=== Basic command ===")
print(result)
assert "hello world" in result
assert "[exit_code: 0]" in result
assert "[cwd:" in result
print("PASS: Basic command with exit code and cwd metadata")
print()

# 2. Non-zero exit code
# Note: 'exit 42' terminates the shell before the CWD probe runs,
# so the exit code is preserved. Simple failures like 'false' get
# masked because the probe's echo succeeds afterward.
result = bash(command="exit 42")
print("=== Non-zero exit code ===")
print(result)
assert "[exit_code: 42]" in result
print("PASS: Non-zero exit code captured")

=== Basic command ===
hello world

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Basic command with exit code and cwd metadata

=== Non-zero exit code ===

[exit_code: 42]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Non-zero exit code captured


In [38]:
# 3. CWD persistence across calls
# First, create a subdirectory inside the sandbox
bash(command="mkdir -p inner_dir")
result1 = bash(command="cd inner_dir && pwd")
print("=== CWD persistence — step 1: cd inner_dir ===")
print(result1)

# Second call should still be in inner_dir
result2 = bash(command="pwd")
print("\n=== CWD persistence — step 2: pwd (should be inner_dir) ===")
print(result2)
assert "inner_dir" in result2
print("PASS: CWD persists across calls\n")

# Reset back to base for subsequent tests
bash(command=f"cd {BASH_DIR}")

=== CWD persistence — step 1: cd inner_dir ===
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox/inner_dir

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox/inner_dir]

=== CWD persistence — step 2: pwd (should be inner_dir) ===
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox/inner_dir

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox/inner_dir]
PASS: CWD persists across calls



'\n[exit_code: 0]\n[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]'

In [39]:
# 4. Sandbox enforcement — cd outside base_path should be blocked
result = bash(command="cd /tmp && pwd")
print("=== Sandbox enforcement: cd /tmp ===")
print(result)
# CWD should still be within BASH_DIR (sandbox prevents escape)
# Verify cwd line shows we're still in the sandbox
cwd_line = [l for l in result.splitlines() if "[cwd:" in l]
if cwd_line:
    # Use resolve() for macOS where /var -> /private/var
    assert str(BASH_DIR.resolve()) in cwd_line[0], f"CWD escaped sandbox: {cwd_line[0]}"
print("PASS: Sandbox prevented escape")
print()

# Verify internal state (resolve both for symlink consistency)
assert bash_tool._cwd.resolve() == BASH_DIR.resolve() or \
    str(bash_tool._cwd).startswith(str(BASH_DIR.resolve())), \
    f"Internal CWD escaped: {bash_tool._cwd}"
print("PASS: Internal _cwd still within sandbox")

=== Sandbox enforcement: cd /tmp ===
/tmp

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Sandbox prevented escape

PASS: Internal _cwd still within sandbox


In [40]:
# 5. dangerouslyDisableSandbox — allows leaving sandbox
result = bash(command="cd /tmp && pwd", dangerouslyDisableSandbox=True)
print("=== dangerouslyDisableSandbox=True ===")
print(result)
assert "/tmp" in result
print("PASS: Sandbox override worked")
print()

# Reset CWD back to sandbox base
bash_tool._cwd = BASH_DIR.resolve()
result = bash(command="pwd")
print("Reset CWD back to sandbox:")
print(result)

=== dangerouslyDisableSandbox=True ===
/tmp

[exit_code: 0]
[cwd: /private/tmp]
PASS: Sandbox override worked

Reset CWD back to sandbox:
/private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]


In [41]:
# 6. Custom timeout
result = bash(command="echo fast", timeout=5000)
print("=== Custom timeout (5000ms) ===")
print(result)
assert "fast" in result
assert "[exit_code: 0]" in result
print("PASS: Custom timeout works\n")

# 7. Output truncation — command producing large output
# Use a BashTool with tiny max_output_chars for testing
small_bash_tool = BashTool(base_path=str(BASH_DIR), max_output_chars=200)
small_bash = small_bash_tool.get_tool()

result = small_bash(command="python3 -c \"print('A' * 2000)\"")
print("=== Output truncation (max_output_chars=200) ===")
print(result)
assert "truncated" in result.lower()
print("PASS: Output truncated")

=== Custom timeout (5000ms) ===
fast

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Custom timeout works

=== Output truncation (max_output_chars=200) ===

... [truncated, showing last 200 chars of 2000 total] ...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Output truncated


In [42]:
# 8. Description parameter — doesn't affect execution
result = bash(command="echo described", description="Print the word 'described'")
print("=== Description parameter ===")
print(result)
assert "described" in result
assert "[exit_code: 0]" in result
print("PASS: Description param does not affect execution\n")

# 9. Agent UUID injection
print("=== Agent UUID injection ===")
print(f"UUID before: {bash_tool.agent_uuid}")
bash_tool.set_agent_uuid("test-uuid-12345")
print(f"UUID after:  {bash_tool.agent_uuid}")
assert bash_tool.agent_uuid == "test-uuid-12345"
# Also verify via __tool_instance__
assert bash.__tool_instance__.agent_uuid == "test-uuid-12345"
print("PASS: UUID injection via set_agent_uuid()\n")

# 10. __tool_schema__ inspection
print("=== Schema inspection ===")
schema = bash.__tool_schema__
print(f"Tool name: {schema['name']}")
print(f"Has description: {bool(schema.get('description'))}")
print(f"Input properties: {list(schema['input_schema']['properties'].keys())}")
assert "command" in schema["input_schema"]["properties"]
assert "timeout" in schema["input_schema"]["properties"]
print("PASS: Schema properly defined")

=== Description parameter ===
described

[exit_code: 0]
[cwd: /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox]
PASS: Description param does not affect execution

=== Agent UUID injection ===
UUID before: None
UUID after:  test-uuid-12345
PASS: UUID injection via set_agent_uuid()

=== Schema inspection ===
Tool name: bash
Has description: True
Input properties: ['command', 'description', 'timeout', 'dangerouslyDisableSandbox']
PASS: Schema properly defined


In [43]:
# 11. Sandbox disabled globally
nosandbox_tool = BashTool(base_path=str(BASH_DIR), sandbox_enabled=False)
nosandbox_bash = nosandbox_tool.get_tool()

result = nosandbox_bash(command="cd /tmp && pwd")
print("=== Sandbox disabled globally ===")
print(result)
assert "/tmp" in result
print("PASS: Sandbox disabled globally allows escape\n")

# 12. CWD to deleted directory — expect reset
deleted_dir = BASH_DIR / "to_be_deleted"
deleted_dir.mkdir(exist_ok=True)

# Create a new bash tool and navigate to the directory
del_bash_tool = BashTool(base_path=str(BASH_DIR))
del_bash = del_bash_tool.get_tool()
del_bash(command=f"cd {deleted_dir}")

# Now delete the directory
shutil.rmtree(str(deleted_dir))

# Next command should detect deleted CWD and reset
result = del_bash(command="pwd")
print("=== CWD to deleted directory ===")
print(result)
# Should reset to base_path
assert str(BASH_DIR) in result
print("PASS: CWD reset after directory deletion")

=== Sandbox disabled globally ===
/tmp

[exit_code: 0]
[cwd: /private/tmp]
PASS: Sandbox disabled globally allows escape

=== CWD to deleted directory ===
Working directory no longer exists. Reset to /private/var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874/bash_sandbox.
PASS: CWD reset after directory deletion


# Cleanup

In [44]:
# Remove all test artifacts
print(f"Cleaning up TEMP_DIR: {TEMP_DIR}")
shutil.rmtree(str(TEMP_DIR))
assert not TEMP_DIR.exists(), f"TEMP_DIR still exists: {TEMP_DIR}"
print("TEMP_DIR removed successfully.")
print("\nAll tests complete. No leftover files.")

Cleaning up TEMP_DIR: /var/folders/z4/yvz5tnyj2lb248t1lscv7x_c0000gn/T/cowork_tool_tests_y7umf874
TEMP_DIR removed successfully.

All tests complete. No leftover files.
